# **Subset CONUS and do a ParFlow Spinup**
### This notebook has two principal sections: 
1. Subset all static inputs from a CONUS run stored in Hydrodata 
2. Load and alter a reference run to spin-up your subset ParFlow domain

### Import the required libraries for step one (subsetting)

In [1]:
import matplotlib.pyplot as plt
import os
from parflow import Run
from parflow.tools.fs import mkdir
from parflow.tools.settings import set_working_directory
from subsettools.subsettools import *
from subsettools.datasets import get_ref_yaml_path

### 1. Define variables to access datasets to subset in Hydrodata and define write paths

#### We will be testing with the Upper Verde watershed for this example
- HUC: 15060202
- Size: 6496 km^2 (ni = 112, nj = 90)   

#### Set your variables to specify which static and climate forcing data you would like to subset in Hydrodata

In [2]:
runname = "spinup_test"

# provide a way to create a subset from the conus domain (huc, lat/lon bbox currently supported)
huc_list = ["15060202"]

# provide information about the datasets you want to access for run inputs using the data catalog
start = "2005-10-01"
wy = 2006
grid = "conus1"
run_ds = "conus1_baseline_mod"
var_ds = "conus1_domain"
P = 1
Q = 1

# Set the directory paths where you want to write your subset files
home = os.path.expanduser("~")
base_dir = os.path.join(home, "subsettools_tutorial")
input_dir = os.path.join(base_dir, "inputs", f"{runname}_{grid}_{wy}WY_spinup")
output_dir = os.path.join(base_dir, "outputs")
static_write_dir = os.path.join(input_dir, "static")
mkdir(static_write_dir)
pf_out_dir = os.path.join(output_dir, f"{runname}_{grid}_{wy}WY_spinup")
mkdir(pf_out_dir)

# set PARFLOW_DIR path to the preferred version of ParFlow
os.environ["PARFLOW_DIR"] = "/home/SHARED/software/parflow/3.10.0"

# reference_run = get_ref_yaml_path(grid)
reference_run = get_ref_yaml_path(grid, "spinup", "solid")
target_runscript = os.path.join(pf_out_dir, runname + ".yaml")

### 2. Get the desired ParFlow x/y bbox from user provided geospatial information 

In [5]:
ij_bounds = huc_to_ij(huc_list=huc_list, grid=grid)
print("ij_bounds returns [imin, jmin, imax, jmax]")
print(f"bounding box: {ij_bounds}")

nj = ij_bounds[3] - ij_bounds[1]
ni = ij_bounds[2] - ij_bounds[0]
print(f"nj: {nj}")
print(f"ni: {ni}")

ij_bounds returns [imin, jmin, imax, jmax]
bounding box: [375, 239, 487, 329]
nj: 90
ni: 112


### 3. Make the mask and solid file
You only do this if you provided a huc or a shapefile where we can extract a mask from the conus tif. Otherwise, the reference run is rewritten to provide a box domain.

In [6]:
create_mask_solid(huc_list=huc_list, grid=grid, write_dir=static_write_dir)

grid is conus1
Wrote mask.pfb
Wrote solidfile.pfsol and mask_vtk.vtk with total z of 500 meters


### 4. Subset the static ParFlow inputs
Two options to subset static inputs. 
1. subset_static(): This function when provided with a variable dataset hosted on hydrodata will subset all static inputs required to do a baseline run from the default argument var_list without the user specify specific files. Pressure is the steady state pressure. If a user would like the override this, they may pass in their own value for var_list and their specifed variables in the target dataset will be subset. 

3. subset_press_init(): This function will write the subset pressure of the last hour in the last day before your start date. If no such pressure file exists in the hydrodata run dataset specifed, no file will be written. The function assumes UTC of zero as the default, but allows the user to override. 

In [7]:
subset_static(ij_bounds, dataset=var_ds, write_dir=static_write_dir)

Wrote slope_x.pfb in specified directory.
Wrote slope_y.pfb in specified directory.
Wrote pf_indicator.pfb in specified directory.
mannings not found in dataset conus1_domain
depth_to_bedrock not found in dataset conus1_domain
Wrote pme.pfb in specified directory.
Wrote ss_pressure_head.pfb in specified directory.


In [9]:
press_init_filename = subset_press_init(
    ij_bounds, dataset=run_ds, date=start, write_dir=static_write_dir, time_zone="UTC"
)
# subset_press_init(ij_bounds, dataset = run_ds, date = start, write_dir = static_write_dir, time_zone = 'America/New_York')
# subset_press_init(ij_bounds, dataset = run_ds, date = start, write_dir = static_write_dir, time_zone = 'US/Central')

Wrote conus1_baseline_mod_2005.09.30:23.00.00_UTC0_press.pfb in specified directory.


### 8. Set up a baseline run from a reference yaml
This function will return the correct template yaml file to do your run based on the grid, if you're doing spin-up and if you're using a solid file with the necessary keys changed to run your subset with selected climate forcing at baseline for one year

In [10]:
target_runscript = edit_runscript_for_subset(
    ij_bounds, runscript_path=reference_run, write_dir=pf_out_dir, runname=runname
)

New runname: spinup_test provided, a new yaml file will be created
No forcing directory provided, run.Solver.CLM.MetFilePath key not set
ComputationalGrid.NY set to 90 and NX to 112
GeomInput.domaininput.InputType detected as SolidFile, no additional keys to change for subset
Updated runscript written to /home/ga6/subsettools_tutorial/outputs/spinup_test_conus1_2006WY_spinup


### 9. Copy over your static files to your run directory
As a seperate function as you may only need to do this once, or you may want to copy subset static files to different run directories 

In [12]:
copy_static_files(read_dir=static_write_dir, write_dir=pf_out_dir)

### 10. Change the file names in your runscript if desired
If you have changed the name of a static input file either from those used in the reference yamls provided on X GitHub repo, or have changed the name of an individual file for an ensemble or other experiment, you can change it with this function by providing the target runscript (yaml or pfidb) and the new file name(s) as an arguments. Only those arguments with a specified file name will be updated

In [14]:
target_runscript = change_filename_values(
    runscript_path=target_runscript,
    write_dir=pf_out_dir,
    init_press=press_init_filename,
)

Initial pressure filename changed to conus1_baseline_mod_2005.09.30:23.00.00_UTC0_press.pfb
Updated runscript written to /home/ga6/subsettools_tutorial/outputs/spinup_test_conus1_2006WY_spinup


### 11. Change processor topolgoy if desired and then distribute your inputs and forcings to match the new topology

In [16]:
dist_run(
    P=P,
    Q=Q,
    runscript_path=target_runscript,
    write_dir=pf_out_dir,
    dist_clim_forcing=False,
)

no forcing dir provided, only distributing static inputs
Distributed mask.pfb with NZ 1
Distributed slope_x.pfb with NZ 1
Distributed slope_y.pfb with NZ 1
Distributed pf_indicator.pfb with NZ 5
Distributed pme.pfb with NZ 5
Distributed ss_pressure_head.pfb with NZ 5
Distributed conus1_baseline_mod_2005.09.30:23.00.00_UTC0_press.pfb with NZ 5


### 12. Do a baseline run.
Load in the yaml run file you've created which is in the same folder as your static forcings and points to your desired Climate forcings. This assumes you do not want to make any changes from the parent model you used (Ex. conus1 baseline) and will run your subset at baseline conditions. Outputs should be almost identical to the parent model at your subset location for the same time period if you make no additional changes.

In [17]:
set_working_directory(pf_out_dir)

# load the specified run script
run = Run.from_definition(target_runscript)
print(f"Loaded run with runname: {run.get_name()}")

# The following line shows timing info that is likely important for interacting with your spin-up run

# run.TimingInfo.BaseUnit = 1.0
# run.TimingInfo.StartCount = 0
# run.TimingInfo.StartTime = 0
# normally set to something like 1000000.0 but shorter for demo purposes
run.TimingInfo.StopTime = 48.0
run.TimingInfo.DumpInterval = 24.0

# run.TimeStep.Type = 'Growth'
# run.TimeStep.InitialStep = .01
# run.TimeStep.GrowthFactor = 1.1
# run.TimeStep.MaxStep = 250.0
# run.TimeStep.MinStep = 0.0001

run.run(working_directory=pf_out_dir)

Loaded run with runname: spinup_test

# ==============================================================================
# ParFlow directory
#  - /home/SHARED/software/parflow/3.10.0
# ParFlow version
#  - 3.10.0
# Working directory
#  - /home/ga6/subsettools_tutorial/outputs/spinup_test_conus1_2006WY_spinup
# ParFlow database
#  - spinup_test.pfidb
# ==============================================================================


# ==============================================================================
# ParFlow ran successfully 💦 💦 💦 
# ==============================================================================



### 13. Restart a parflow run
If you need to restart a run to complete a spinup, transient run, etc then you can run the following to make the necessary updates to your runscript and then overwrite it in place

In [ ]:
# restart_run(target_runscript)